<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

## 4D Analytics using the New York City Taxi dataset
</header>

### Introduction

We will explore the New York Taxi demo dataset and get familiar with Vantage temporal and geospatial functions.

## Connect Vantage and explore the dataset

We will connect the Transcend system and locate our demo dataset for this exercise.

In [ ]:
%connect local

We have two tables: the taxi trip details and the fares for each trip

In [ ]:
SELECT top 10 * from TRNG_NYCTaxi.trip;

In [ ]:
SELECT top 10 * from TRNG_NYCTaxi.trip_fare;

In [ ]:
sel min(pickup_datetime), max(dropoff_datetime) from TRNG_NYCTaxi.trip;

## Warmup: temporal algebra

Let's warmup with a bit of temporal algebra and get familiar with poeriod types and operators.

Count the number of taxi pickups between 10:30 and 10:45 on Nov 10th
![](contains.png)

In [1]:
sel count(1)
from TRNG_NYCTaxi.trip  
--where pickup_datetime between '2013-11-10 10:30:00' and '2013-11-10 10:45:00'
--Here is an alternative to the above using period data type a
where period
	(
		'2013-11-10 10:30:00' (timestamp), 
		'2013-11-10 10:45:00' (timestamp)
	) 
	CONTAINS pickup_datetime 

ERROR: No active connection

How many rides started and ended between 10:30 and 10:45 on Nov 10th?
![](contains-period.png)

In [ ]:
SELECT count(1)
FROM TRNG_NYCTaxi.trip  
WHERE  pickup_datetime < dropoff_datetime
and period
	(
		'2013-11-10 10:30:00' (timestamp), 
		'2013-11-10 10:45:00' (timestamp)
	) 
	contains period(pickup_datetime, dropoff_datetime);

How many passengers were in a taxi between 10:30 and 10:45 on Nov 10th?
![](overlap.png)

In [ ]:
SELECT sum(passenger_count)
FROM TRNG_NYCTaxi.trip  
WHERE pickup_datetime < dropoff_datetime
and period
	(
		'2013-11-10 10:30:00' (timestamp), 
		'2013-11-10 10:45:00' (timestamp)
	) 
	overlaps period(pickup_datetime, dropoff_datetime)
;

Let's assume we run the fleet of the 3 taxis identifiable by medallion starting with ‘007’. The fleet is considered active if at least 1 taxi is driving. 

For how long has the fleet been “active” on November 10th?
![](normalize.png)

Let's first try to normalize overlaping perdiods.

In [ ]:
SELECT 
period(pickup_datetime, dropoff_datetime) as norm_per
from TRNG_NYCTaxi.trip  
where (pickup_datetime (date))='2013-11-10'
and medallion like '007%'
order by 1;

And now sum up the duration of each period:

In [ ]:
SELECT sum(interval(norm_per) hour(2) to minute)
from
(
	SELECT normalize
	period(pickup_datetime, dropoff_datetime) as norm_per
	from TRNG_NYCTaxi.trip  
	where (pickup_datetime (date))='2013-11-10'
	and medallion like '007%'
) n
order by 1;

So the inner query above gives us a normalized timeline (one row represents a period when my fleet is active, until it is not anymore).

Such timeline, series of time periods, can be expanded again, into a series of fixed frequency periods.
In this case we will check every hour if our fleet is active or not.

In [ ]:
sel begin(expd) hour_check, 
a.norm_per activity_period
from
(
	sel normalize
	period(pickup_datetime, dropoff_datetime) as norm_per
	from TRNG_NYCTaxi.trip  
	where (pickup_datetime (date))='2013-11-10'
	and medallion like '007%'
) a
EXPAND ON norm_per AS expd BY ANCHOR ANCHOR_HOUR
order by 1;

## Time series analysis

Let's now move on to time series transformations and analytics.

### Time series analysis

How many passengers are being picked up by hour in november?

In [ ]:
sel top 72
$TD_TIMECODE_RANGE 
,begin($TD_TIMECODE_RANGE) time_bucket_start --(timestamp, format 'YYYY-MM-DDBHH:MI:SS') time_bucket_start
,sum(passenger_count) passenger_count
from TRNG_NYCTaxi.trip 
where extract(month from pickup_datetime)=11
group by time(hours(1))
USING TIMECODE(pickup_datetime)
order by 1;

It's about time to add some visual element...

In [ ]:
%chart x=time_bucket_start, y=passenger_count, title=Passenger pickup by hour

How many passengers are being picked up and what is the average trip duration by vendor every 30mn in November?

In [ ]:
sel top 96
$TD_TIMECODE_RANGE 
,vendor_id
,sum(passenger_count)
,avg(trip_time_in_secs)
from TRNG_NYCTaxi.trip
group by time(minutes(15) and vendor_id)
USING TIMECODE(pickup_datetime)
where extract(month from pickup_datetime)=11
order by 1,2;

Let's call this a view

In [ ]:
replace view NYC_taxi_trip_ts as
sel
$TD_TIMECODE_RANGE time_bucket_per
,vendor_id
,sum(passenger_count) passenger_cnt
,sum(trip_time_in_secs) avg_trip_time_in_secs
from TRNG_NYCTaxi.trip 
group by time(minutes(15) and vendor_id)
USING TIMECODE(pickup_datetime)
where extract(month from pickup_datetime)=11;

### Moving Averages

Let's calculate a 2 hours minutes moving average on our 15-minutes time series. 2 hour is 8 * 15 minutes periods.

In [ ]:
SELECT * FROM MovingAverage (
  ON NYC_taxi_trip_ts PARTITION BY vendor_id ORDER BY time_bucket_per
  USING
  MAvgType ('S')
  WindowSize (8)
  TargetColumns ('passenger_cnt')
) AS dt 
where begin(time_bucket_per)(date) = '2013-11-10'
ORDER BY vendor_id, time_bucket_per;

We can use this to identify demand trends for each time period. A basic way to do this would be to check if the current pickup count is above (rising) or below (reducing) our moving average.

In [ ]:
SELECT time_bucket_per, vendor_id, passenger_cnt, trend, case when passenger_cnt - passenger_cnt_smavg > 0 then '+' else '-' end trend
FROM MovingAverage (
  ON NYC_taxi_trip_ts PARTITION BY vendor_id ORDER BY time_bucket_per
  USING
  MAvgType ('S')
  WindowSize (8)
  TargetColumns ('passenger_cnt')
) AS dt 
where begin(time_bucket_per)(date) = '2013-11-10'
ORDER BY vendor_id, time_bucket_per;

We can use this to compare how vendors are doing.

In [ ]:
sel dt.*
from 
(
	SELECT time_bucket_per, vendor_id, passenger_cnt, case when passenger_cnt - passenger_cnt_smavg > 0 then '+' else '-' end trend
	FROM MovingAverage (
	  ON NYC_taxi_trip_ts PARTITION BY vendor_id ORDER BY time_bucket_per
	  USING
	  MAvgType ('S')
	  WindowSize (8)
	  TargetColumns ('passenger_cnt')
	) AS dt 
	where begin(time_bucket_per)(date) = '2013-11-10'
) dt
PIVOT(
    MAX(passenger_cnt) as passenger_cnt, MAX(trend) as trend FOR vendor_id IN ('CMT', 'VTS')
) dt
order by 1;

### Geospatial analysis

Let's define a few landmarks.

In [ ]:
CREATE VOLATILE TABLE dim_geo_locations
     (
      location VARCHAR(100),
      Lat FLOAT,
      Lon FLOAT,
      geo_point SYSUDTLIB.ST_GEOMETRY(16776192) INLINE LENGTH 9920)
PRIMARY INDEX ( location )
ON COMMIT PRESERVE ROWS;

In [ ]:
insert into dim_geo_locations values('Columbia University',40.81,-73.96,'POINT(40.81 -73.96)');
insert into dim_geo_locations values('Empire State Building',40.75,-73.99,'POINT(40.75 -73.99)');
insert into dim_geo_locations values('Grand Central Station',40.75,-73.98,'POINT(40.75 -73.98)');
insert into dim_geo_locations values('JFK Airport',40.64,-73.79,'POINT(40.64 -73.79)');
insert into dim_geo_locations values('Madison Square Garden',40.75,-73.99,'POINT(40.75 -73.99)');
insert into dim_geo_locations values('New York Stock Exchange',40.71,-74.01,'POINT(40.71 -74.01)');
insert into dim_geo_locations values('Times Square',40.76,-73.99,'POINT(40.76 -73.99)');
insert into dim_geo_locations values('United Nations HQ',40.75,-73.97,'POINT(40.75 -73.97)');
insert into dim_geo_locations values('Yankee Stadium',40.83,-73.93,'POINT(40.83 -73.93)');

We are casting coordinates as a ST_GEOMETRY type. Here are the coordinates for the Yankee Stadium, and this is a point:

In [ ]:
sel cast('POINT(40.75 -73.97)' as ST_GEOMETRY)

Let's filter amd label the rides starting within 1km from a given landmark.

In [ ]:
sel
l.location
,cast('POINT('||trim(r.pickup_latitude (Decimal(15,6)))||' '||trim(r.pickup_longitude (Decimal(15,6)))||')' as ST_GEOMETRY) pickup_point
,r.*
from TRNG_NYCTaxi.trip r
join dim_geo_locations l
    on pickup_point.ST_SphericalDistance(l.geo_point)<1000
where (r.pickup_datetime (date)) = '2013-11-10'
;

What is the number of pickup at each location throughout the month?

In [ ]:
sel
$TD_TIMECODE_RANGE time_bucket_per
,l.location
,count(1) pickup_cnt
from TRNG_NYCTaxi.trip r
join dim_geo_locations l
	on cast('POINT('||trim(r.pickup_latitude (Decimal(15,6)))||' '||trim(r.pickup_longitude (Decimal(15,6)))||')' as ST_GEOMETRY).ST_SphericalDistance(l.geo_point)<1000
group by time(minutes(15) and l.location)
USING TIMECODE(pickup_datetime)
where extract(month from pickup_datetime)=11
order by 2,1;

What is the average demand at each location throughout the day, based on November data?

In [ ]:
sel *
from 
(
	sel
	begin($TD_TIMECODE_RANGE) (time) timeOfDay
	,l.location
	,count(1) pickup_cnt
	from TRNG_NYCTaxi.trip r
	join dim_geo_locations l
		on cast('POINT('||trim(r.pickup_latitude (Decimal(15,6)))||' '||trim(r.pickup_longitude (Decimal(15,6)))||')' as ST_GEOMETRY).ST_SphericalDistance(l.geo_point)<1000
	group by time(minutes(15) and l.location)
	USING TIMECODE(pickup_datetime)
	where extract(month from pickup_datetime)=11
) AS dt 
PIVOT(
    avg(pickup_cnt) FOR location IN (sel distinct location from dim_geo_locations)
) dt
order by 1;

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2020 Teradata. All Rights Reserved</footer>